In [168]:
#function to test models 
import nest_asyncio
nest_asyncio.apply()

In [169]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.dataloader.loader import DataLoader

In [170]:
k = DataLoader('../scripts/model_load/loader_y.yaml')

In [171]:
k.data_dict

{'data': {'from_datasets': 'CIFAR10',
  'from_file': False,
  'work_dir': '/home/malav/mb_pytorch/data/',
  'file': '/home/malav/mb_pytorch/data/csv/file_data.csv',
  'use_img_dir': True,
  'img_dir': '/home/malav/mb_pytorch/data/images1/',
  'thresholding_pd': 0},
 'train_params': {'batch_size': 128,
  'num_workers': 4,
  'pin_memory': True,
  'shuffle': True,
  'seed': 42},
 'test_params': {'batch_size': 128,
  'num_workers': 4,
  'pin_memory': True,
  'shuffle': True,
  'seed': 42},
 'transforms_list': {'transform': True,
  'resize': {'val': False, 'args': {'size': [256, 256]}},
  'random_crop': {'val': False, 'args': {'size': [224, 224]}},
  'random_horizontal_flip': {'val': True, 'args': {'p': 0.5}},
  'random_vertical_flip': {'val': False, 'args': {'p': 0.5}},
  'random_rotation': {'val': True, 'args': {'degrees': 90}},
  'random_grayscale': {'val': False, 'args': {'p': 0.1}},
  'random_color_jitter': {'val': True,
   'args': {'brightness': 0.4,
    'contrast': 0.4,
    'saturati

In [172]:
model_data = ModelLoader(k.data_dict['model'])

In [173]:
model_data.get_model()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [174]:
train_loader,val_loader,_,_ = k.data_load()

Files already downloaded and verified
Files already downloaded and verified


In [175]:
from mb_pytorch.training.train_params import train_helper

In [176]:
model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [177]:
print(loss_attr())
print(optimizer_attr)
print(optimizer_dict)
print(scheduler_attr)
print(scheduler_dict)

CrossEntropyLoss()
<class 'torch.optim.adam.Adam'>
{'lr': 0.001, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}
<class 'torch.optim.lr_scheduler.StepLR'>
{'step_size': 3, 'gamma': 0.2}


In [178]:
import tqdm
from torch.utils.tensorboard import SummaryWriter
import os

In [179]:
all_data = k.data_dict
all_data['data']

{'from_datasets': 'CIFAR10',
 'from_file': False,
 'work_dir': '/home/malav/mb_pytorch/data/',
 'file': '/home/malav/mb_pytorch/data/csv/file_data.csv',
 'use_img_dir': True,
 'img_dir': '/home/malav/mb_pytorch/data/images1/',
 'thresholding_pd': 0}

In [180]:

#optimizer_dict['lr'] = scheduler_attr.get_last_lr()[0]
optimizer =optimizer_attr(model_data.parameters(),**optimizer_dict)
scheduler = scheduler_attr(optimizer,**scheduler_dict)

best_val_loss = float('inf')

path_logs = os.path.join(all_data['data']['work_dir'], 'logs')
writer = SummaryWriter(log_dir=path_logs)

In [181]:
all_model_data= k.data_dict['model']
all_model_data

{'load_model': False,
 'use_torchvision_models': True,
 'model_name': 'resnet',
 'model_version': '50',
 'model_custom': 'mb_pytorch.models.unet_models',
 'model_path': '/home/malav/mb_pytorch/models/',
 'model_optimizer': 'Adam',
 'model_loss': 'CrossEntropyLoss',
 'model_scheduler': 'StepLR',
 'model_pretrained': False,
 'model_num_classes': 10,
 'model_initialization': 'None',
 'model_epochs': 50,
 'model_train_parameters': {'Adam': {'lr': 0.001,
   'betas': [0.9, 0.999],
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False},
  'SGD': {'lr': 0.01,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False},
  'RMSprop': {'lr': 0.01,
   'alpha': 0.99,
   'eps': 1e-08,
   'weight_decay': 0,
   'momentum': 0,
   'centered': False},
  'cross_entropy': {'weight': 'None',
   'size_average': 'None',
   'ignore_index': -100,
   'reduce': 'None',
   'reduction': 'mean'},
  'mse': {'size_average': 'None', 'reduce': 'None', 'reduction': 'mean'},
  'StepLR': {'step_

In [182]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [183]:
from mb_utils.src.logging import logger
logger=logger

In [184]:
from itertools import islice
train_loader10 = islice(train_loader, 50)
val_loader10 = islice(val_loader, 50)

In [185]:
for i in tqdm.tqdm(range(all_model_data['model_epochs'])):
    ##train loop
    model_data.train()
    train_loss = 0
    for j,(x,y) in enumerate(train_loader10):
        x,y = x.to(device),y.to(device)
        optimizer.zero_grad()
        y_pred = model_data(x)
        current_loss = loss_attr()(y_pred,y)
        current_loss.backward()    
        optimizer.step()
        train_loss += current_loss.item()
        logger.info(f'Epoch {i+1} - Batch {j+1} - Train Loss: {current_loss.item()}')
    
    avg_train_loss = train_loss / len(train_loader)
    logger.info(f'Epoch {i+1} - Train Loss: {avg_train_loss}')
    
    writer.add_scalar('Loss/train', avg_train_loss, global_step=i)
        
    if scheduler is not None:
        scheduler.step()
    
    for name, param in model_data.named_parameters():
        writer.add_histogram(name, param, global_step=i)
        
    #get grad cam images
        
    #validation loop
    val_loss = 0
    val_acc = 0
    num_samples = 0
    
    model_data.eval()
    with torch.no_grad():
        for x_val, y_val in val_loader10:
            x_val, y_val = x_val.to(device), y_val.to(device)
            output = model_data(x_val)
            val_loss += loss_attr()(output, y_val).item() * x_val.size(0)
            _, preds = torch.max(output, 1)
            val_acc += torch.sum(preds == y_val.data)
            num_samples += x_val.size(0)
            logger.info(f'Epoch {i+1} - Batch {j+1} - Val Loss: {val_loss}')
        
        if num_samples == 0:
            num_samples = 1
        val_loss /= num_samples
        val_acc = val_acc / num_samples
        logger.info(f'Epoch {i+1} - Val Loss: {val_loss}', f'Epoch {i+1} - Val Acc: {val_acc}')
    
    writer.add_scalar('Loss/val', val_loss, global_step=i)
    writer.add_scalar('Accuracy/val', val_acc, global_step=i)
    
    # save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model_data.state_dict()

        path = os.path.join(all_data['data']['work_dir'], 'best_model.pth')
        torch.save(best_model, path)
        logger.info(f'Epoch {i+1} - Best Model Saved')
        
    model_data.train()
        


  0%|          | 0/10 [00:00<?, ?it/s]Tue 01:14:24,219 INF Epoch 1 - Batch 1 - Train Loss: 7.467926502227783                                                               
Tue 01:14:26,067 INF Epoch 1 - Batch 2 - Train Loss: 4.360098361968994                                                               
Tue 01:14:27,861 INF Epoch 1 - Batch 3 - Train Loss: 3.3480334281921387                                                              
Tue 01:14:29,605 INF Epoch 1 - Batch 4 - Train Loss: 2.9955832958221436                                                              
Tue 01:14:31,377 INF Epoch 1 - Batch 5 - Train Loss: 2.969878911972046                                                               
Tue 01:14:33,181 INF Epoch 1 - Batch 6 - Train Loss: 3.361004114151001                                                               
Tue 01:14:35,016 INF Epoch 1 - Batch 7 - Train Loss: 3.0055198669433594                                                              
Tue 01:14:36,910 INF Epo

In [187]:
from mb_pytorch.utils.extra_utils import feature_view

In [189]:
for l in train_loader10:
    x,y = l
    break

In [191]:
x[0].shape

torch.Size([3, 32, 32])

In [195]:
feature_view(x[0],model_data,layer_names=['conv1','conv2','conv3'])

TypeError: slice indices must be integers or None or have an __index__ method